# GPU-Jupyter

This Jupyterlab Instance is connected to the GPU via CUDA drivers. In this notebook, we test the installation and perform some basic operations on the GPU.

## Test GPU connection

#### Using the following command, your GPU type and its NVIDIA-SMI driver version should be listed:

In [1]:
!nvidia-smi

Mon Jan 18 13:03:00 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 450.51.05    Driver Version: 450.51.05    CUDA Version: 11.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  On   | 00000000:1A:00.0 Off |                    0 |
| N/A   35C    P0    53W / 300W |    943MiB / 16160MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
|   1  Tesla V100-SXM2...  On   | 00000000:B3:00.0 Off |                    0 |
| N/A   

#### Now, test if PyTorch can access the GPU via CUDA:

In [2]:
import torch
torch.cuda.is_available()

True

In [3]:
import tensorflow as tf
from tensorflow.python.client import device_lib
print(tf.config.list_physical_devices('XLA_GPU'))
device_lib.list_local_devices()

[]


[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 18052359506885804551,
 name: "/device:GPU:0"
 device_type: "GPU"
 memory_limit: 14071056320
 locality {
   bus_id: 1
   links {
     link {
       device_id: 1
       type: "StreamExecutor"
       strength: 1
     }
   }
 }
 incarnation: 449937755150116278
 physical_device_desc: "device: 0, name: Tesla V100-SXM2-16GB, pci bus id: 0000:1a:00.0, compute capability: 7.0",
 name: "/device:GPU:1"
 device_type: "GPU"
 memory_limit: 14071056320
 locality {
   bus_id: 2
   numa_node: 1
   links {
     link {
       type: "StreamExecutor"
       strength: 1
     }
   }
 }
 incarnation: 2613653959801675174
 physical_device_desc: "device: 1, name: Tesla V100-SXM2-16GB, pci bus id: 0000:b3:00.0, compute capability: 7.0"]

In [4]:
from __future__ import print_function
import numpy as np
import torch
a = torch.rand(5, 3)
a

tensor([[0.1044, 0.1919, 0.1278],
        [0.4688, 0.7709, 0.5211],
        [0.6677, 0.1942, 0.6678],
        [0.8274, 0.3681, 0.9967],
        [0.6043, 0.5823, 0.3560]])

## Performance test

#### Now we want to know how much faster a typical operation is using GPU. Therefore we do the same operation in numpy, PyTorch and PyTorch with CUDA. The test operation is the calculation of the prediction matrix that is done in a linear regression.

### 1) Numpy

In [5]:
x = np.random.rand(10000, 256)

In [6]:
%%timeit
H = x.dot(np.linalg.inv(x.transpose().dot(x))).dot(x.transpose())

276 ms ± 9.97 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


### 2) PyTorch

In [7]:
x = torch.rand(10000, 256)

In [8]:
%%timeit
# Calculate the projection matrix of x on the CPU
H = x.mm( (x.t().mm(x)).inverse() ).mm(x.t())

82.1 ms ± 1.85 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


### 3) PyTorch on GPU via CUDA

In [9]:
# let us run this cell only if CUDA is available
# We will use ``torch.device`` objects to move tensors in and out of GPU
if torch.cuda.is_available():
    device = torch.device("cuda")          # a CUDA device object
    x = torch.rand(10000, 256, device=device) # directly create a tensor on GPU
    y = x.to(device)                       # or just use strings ``.to("cuda")``
    print(x[0:5, 0:5])
    print(y.to("cpu", torch.double)[0:5, 0:5])

tensor([[0.2854, 0.3384, 0.6473, 0.0433, 0.5640],
        [0.3960, 0.0449, 0.6597, 0.5347, 0.8402],
        [0.0048, 0.9231, 0.0311, 0.2545, 0.0409],
        [0.6506, 0.8651, 0.7558, 0.1086, 0.8135],
        [0.1083, 0.0039, 0.6049, 0.3596, 0.1359]], device='cuda:0')
tensor([[0.2854, 0.3384, 0.6473, 0.0433, 0.5640],
        [0.3960, 0.0449, 0.6597, 0.5347, 0.8402],
        [0.0048, 0.9231, 0.0311, 0.2545, 0.0409],
        [0.6506, 0.8651, 0.7558, 0.1086, 0.8135],
        [0.1083, 0.0039, 0.6049, 0.3596, 0.1359]], dtype=torch.float64)


In [10]:
%%timeit
# Calculate the projection matrix of x on the GPU
H = x.mm( (x.t().mm(x)).inverse() ).mm(x.t())

11.4 ms ± 28.8 µs per loop (mean ± std. dev. of 7 runs, 1 loop each)


## Exhaustive Testing on GPU

In [11]:
# let us run this cell only if CUDA is available
# We will use ``torch.device`` objects to move tensors in and out of GPU
import torch
if torch.cuda.is_available():
    device = torch.device("cuda")          # a CUDA device object
    x = torch.rand(10000, 10, device=device) # directly create a tensor on GPU

In [12]:
if torch.cuda.is_available():
    y = x.to(device)                       # or just use strings ``.to("cuda")``
    print(x[0:5, 0:5])

tensor([[0.1101, 0.7887, 0.0641, 0.1327, 0.1681],
        [0.7914, 0.7248, 0.7731, 0.2662, 0.4908],
        [0.2451, 0.3568, 0.4006, 0.2099, 0.5212],
        [0.6195, 0.5120, 0.5212, 0.7321, 0.2272],
        [0.2374, 0.4540, 0.0868, 0.9393, 0.1561]], device='cuda:0')


In [13]:
if torch.cuda.is_available():
    # Here is the memory of the GPU a border. 
    # A matrix with 100000 lines requires 37 GB, but only 8 GB are available.
    H = x.mm( (x.t().mm(x)).inverse() ).mm(x.t())

In [14]:
if torch.cuda.is_available():
    print(H[0:5, 0:5])

tensor([[ 6.4681e-04, -1.5392e-05,  3.3608e-04,  2.1025e-04,  8.0912e-05],
        [-1.5392e-05,  5.0718e-04, -1.1769e-04, -2.3084e-05, -2.3264e-04],
        [ 3.3608e-04, -1.1769e-04,  6.9678e-04,  2.2663e-04, -1.8900e-04],
        [ 2.1025e-04, -2.3084e-05,  2.2663e-04,  6.0036e-04,  2.7787e-04],
        [ 8.0912e-05, -2.3264e-04, -1.8900e-04,  2.7787e-04,  1.4208e-03]],
       device='cuda:0')


In [15]:
if torch.cuda.is_available():
    # This operation is difficult, as an symmetric matrix is transferred 
    # back to the CPU. Is possible up to 30000 rows.
    print(H.to("cpu", torch.double)[0:5, 0:5])

tensor([[ 6.4681e-04, -1.5392e-05,  3.3608e-04,  2.1025e-04,  8.0912e-05],
        [-1.5392e-05,  5.0718e-04, -1.1769e-04, -2.3084e-05, -2.3264e-04],
        [ 3.3608e-04, -1.1769e-04,  6.9678e-04,  2.2663e-04, -1.8900e-04],
        [ 2.1025e-04, -2.3084e-05,  2.2663e-04,  6.0036e-04,  2.7787e-04],
        [ 8.0912e-05, -2.3264e-04, -1.8900e-04,  2.7787e-04,  1.4208e-03]],
       dtype=torch.float64)
